In [1]:
from tqdm.notebook import tqdm

types = {
    0: ('nolabel', "#000000"),
    1: ('neoplastic', "#CD5C5C"),
    2: ('inflammatory', "#9FE2BF"),
    3: ('connective', "#6495ED"),
    4: ('necrotic', "#DFFF00"),
    5: ('non-neoplastic', "#FFBF00")
}

count = {
    'nolabel':0,
    'neoplastic':0,
    'inflammatory':0,
    'connective':0,
    'necrotic':0,
    'non-neoplastic':0
}

start_of_document = '''<?xml version="1.0"?>
<ASAP_Annotations>
\t<Annotations>
'''

end_of_document = '''
\t</Annotations>
</ASAP_Annotations>
'''

def json2xml(annotation):
    type,color = types[annotation['type']]
    if not type in [
        # 'neoplastic',
        'inflammatory'
    ]:
        return ''

    count[type] +=1
    name= f'{type} {count[type]}'
    
    start_of_annotation = f'''\t\t<Annotation Name="{name}" Type="Polygon" PartOfGroup="None" Color="{color}">
    \t\t<Coordinates>
    '''

    coordinates=''
    for ix, (x,y) in enumerate(annotation['contour']):
        if ix == 0:
            coordinate = f'\t\t\t<Coordinate Order="{ix}" X="{x*2}" Y="{y*2}" />\n'   
        else:
            coordinate = f'\t\t\t\t<Coordinate Order="{ix}" X="{x*2}" Y="{y*2}" />\n'
        coordinates += coordinate

    end_of_annotation = '''\t\t\t</Coordinates>
    \t</Annotation>'''

    return start_of_annotation + coordinates + end_of_annotation



In [2]:
def json2bbox(annotation):
    type,color = types[annotation['type']]
    if not type in [
        # 'neoplastic',
        'inflammatory'
    ]:
        return ''

    count[type] +=1
    name= f'{type} {count[type]}'
    
    start_of_annotation = f'''\t\t<Annotation Name="{name}" Type="Rectangle" PartOfGroup="None" Color="{color}">
    \t\t<Coordinates>
    '''

    (x1, y1), (x2, y2) = cell['bbox']
    coordinates=f'''\t\t\t<Coordinate Order="0" X="{x1 * 2}" Y="{y1 * 2}" />
	\t\t\t<Coordinate Order="1" X="{x2*2}" Y="{y1*2}" />
	\t\t\t<Coordinate Order="2" X="{x2*2}" Y="{y2*2}" />
	\t\t\t<Coordinate Order="3" X="{x1*2}" Y="{y2*2}" />
    '''

    end_of_annotation = '''\t\t</Coordinates>
    \t</Annotation>'''

    return start_of_annotation + coordinates + end_of_annotation

In [3]:
from pathlib import Path
import json


path = Path(r'/data/data/pathology/isala/json/PREM-IS-005_d5fa80a160 I10 HE1 - 2022-03-24 19.45.44.json')

with open(path) as f:
    annotation = json.load(f)

cells=''
for _, cell in tqdm(annotation['nuc'].items()):
    cells += json2bbox(cell)

xml = start_of_document + cells + end_of_document

with open(Path('/data/data/pathology/isala/xml/') / (path.stem + '.xml'), 'w') as f:
    f.write(xml)

  0%|          | 0/938487 [00:00<?, ?it/s]